In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext line_profiler

In [ ]:
from operator import itemgetter
from functools import partial
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import time
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 10 * 2.54})
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}"
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import jax
import jax.numpy as jnp
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update("jax_enable_x64", True)
import diffrax
import equinox as eqx
import optax

from haiku import PRNGSequence

In [ ]:
from dmpe.utils.signals import aprbs
from dmpe.evaluation.plotting_utils import (
    plot_sequence, append_predictions_to_sequence_plot, plot_sequence_and_prediction, plot_model_performance,
    plot_2d_kde_as_contourf, plot_2d_kde_as_surface, plot_feature_combinations
)

In [ ]:
from dmpe.related_work.algorithms import excite_with_iGOATS

In [ ]:
from exciting_environments.pmsm.pmsm_env import PMSM

In [ ]:
class ExcitingPMSM(PMSM):

    def generate_observation(self, system_state, env_properties):
        physical_constraints = env_properties.physical_constraints

        eps = system_state.physical_state.epsilon
        cos_eps = jnp.cos(eps)
        sin_eps = jnp.sin(eps)

        obs = jnp.hstack(
            (
                (system_state.physical_state.i_d + (physical_constraints.i_d * 0.5))
                / (physical_constraints.i_d * 0.5),
                system_state.physical_state.i_q / physical_constraints.i_q,
            )
        )
        return obs

    def init_state(self, env_properties, rng=None, vmap_helper=None):
        """Returns default initial state for all batches."""
        phys = self.PhysicalState(
            u_d_buffer=0.0,
            u_q_buffer=0.0,
            epsilon=0.0,
            i_d=-env_properties.physical_constraints.i_d / 2,
            i_q=0.0,
            torque=0.0,
            omega_el=2 * jnp.pi * 3 * 1000 / 60,
        )
        subkey = jnp.nan
        additions = None  # self.Optional(something=jnp.zeros(self.batch_size))
        ref = self.PhysicalState(
            u_d_buffer=jnp.nan,
            u_q_buffer=jnp.nan,
            epsilon=jnp.nan,
            i_d=jnp.nan,
            i_q=jnp.nan,
            torque=jnp.nan,
            omega_el=jnp.nan,
        )
        return self.State(physical_state=phys, PRNGKey=subkey, additions=additions, reference=ref)

batch_size = 1

env = ExcitingPMSM(
    batch_size=batch_size,
    saturated=True,
    static_params={
        "p": 3,
        "r_s": 15e-3,
        "l_d": jnp.nan,
        "l_q": jnp.nan,
        "psi_p": jnp.nan,
        "deadtime": 0,
    },
    solver=diffrax.Euler(),
)

from dmpe.excitation.excitation_utils import soft_penalty

def PMSM_penalty(observations, actions, penalty_order=2):

    action_penalty = soft_penalty(actions, a_max=1, penalty_order=1)

    physical_i_d = observations[..., 0] * (env.env_properties.physical_constraints.i_d * 0.5) - (
        env.env_properties.physical_constraints.i_d * 0.5
    )
    physical_i_q = observations[..., 1] * env.env_properties.physical_constraints.i_q

    a = physical_i_d / 250
    b = physical_i_q / 250

    obs_penalty = jax.nn.relu(a**2 + b**2 - 0.9)
    obs_penalty = jnp.sum(obs_penalty)
    i_d_penalty = jnp.sum(jax.nn.relu(a))

    return (obs_penalty + i_d_penalty + action_penalty) * 1e3

In [ ]:
# setup PRNG
seed = 1

key = jax.random.PRNGKey(seed=seed)

data_key, model_key, loader_key, expl_key, key = jax.random.split(key, 5)
data_rng = PRNGSequence(data_key)

obs, state = env.reset(env.env_properties)

n_steps = 5
actions = jnp.concatenate([aprbs(n_steps, batch_size, 1, 1, next(data_rng)), aprbs(n_steps, batch_size, 1, 1, next(data_rng))], axis=-1)

actions = jnp.ones((batch_size, n_steps, 1)) * 1
actions = jnp.concatenate([jnp.zeros((batch_size, n_steps, 1)), actions], axis=-1)[0]


observations = [obs]

for i in range(actions.shape[0]):
   
    obs, state = env.step(state, actions[i,:], env.env_properties)
    observations.append(obs)


plot_sequence(np.stack(observations), actions, env.tau, obs_labels=env.obs_description[:2], action_labels=['u_d', 'u_q'])

In [ ]:
h = 4
a = 4

alg_params = dict(
    prediction_horizon=h,
    application_horizon=a,
    bounds_amplitude=(-1, 1),
    bounds_duration=(1, 50),
    population_size=50,
    n_generations=50,
    featurize=lambda x: x,
    rng=None,
    compress_data=False,
    compression_target_N=None,
    compression_feat_dim=None,
    compression_dist_th=None,
    penalty_function=PMSM_penalty,
)

exp_params = dict(
    n_timesteps=5_000,
    seed=int(seed),
    alg_params=alg_params,
    env_params=None,
)

# run excitation algorithm
observations, actions = excite_with_iGOATS(
    n_timesteps=exp_params["n_timesteps"],
    env=env,
    prediction_horizon=alg_params["prediction_horizon"],
    application_horizon=alg_params["application_horizon"],
    bounds_amplitude=alg_params["bounds_amplitude"],
    bounds_duration=alg_params["bounds_duration"],
    population_size=alg_params["population_size"],
    n_generations=alg_params["n_generations"],
    featurize=alg_params["featurize"],
    rng=np.random.default_rng(seed),
    compress_data=alg_params["compress_data"],
    compression_target_N=alg_params["compression_target_N"],
    compression_feat_dim=alg_params["compression_feat_dim"],
    compression_dist_th=alg_params["compression_dist_th"],
    penalty_function=alg_params["penalty_function"],
    plot_subsequences=True,
)

In [ ]:
%debug

In [ ]:
raise

In [ ]:
observations = np.concatenate(observations)
actions = np.concatenate(actions)

np.save("results/pmsm_igoats_observations.npy", observations)
np.save("results/pmsm_igoats_actions.npy", actions)

In [ ]:
observations = np.load("results/pmsm_igoats_observations.npy")
actions = np.load("results/pmsm_igoats_actions.npy")

In [ ]:
plot_sequence(observations, actions, env.tau, obs_labels=env.obs_description[:2], action_labels=env.action_description)

In [ ]:
physical_i_d = observations[..., 0] * (env.env_properties.physical_constraints.i_d * 0.5) - (env.env_properties.physical_constraints.i_d * 0.5)
physical_i_q = observations[..., 1] * env.env_properties.physical_constraints.i_q

In [ ]:
fig, axs = plot_sequence(
    observations=jnp.vstack([physical_i_d, physical_i_q]).T,
    actions=(actions * jnp.hstack([env.env_properties.action_constraints.u_d, env.env_properties.action_constraints.u_q])),
    tau=env.tau,
    obs_labels=["i_d", "i_q"],
    action_labels=["u_d", "u_q"]
)
# t = jnp.linspace(0, observations.shape[0] - 1, observations.shape[0]) * env.tau
# axs[0].plot(t, np.ones(observations.shape[0]) * env.env_properties.physical_constraints.i_d)
# axs[0].plot(t, -np.ones(observations.shape[0]) * env.env_properties.physical_constraints.i_d)

axs[1].set_xlim(-250,0)
axs[1].set_ylim(-250, 250)
# t = t[:-1]
# axs[2].plot(t, np.ones(actions.shape[0]) * env.env_properties.action_constraints.u_d)
# axs[2].plot(t, -np.ones(actions.shape[0]) * env.env_properties.action_constraints.u_d)

plt.savefig("learning_dmpe_200vdc.png")

In [ ]:
fig = plot_feature_combinations(
    jnp.concatenate([observations, actions], axis=-1),
    labels=["$\\tilde{i}_d$", "$\\tilde{i}_q$", "$\\tilde{u}_d$", "$\\tilde{u}_q$"],
    mode="contourf"
);
plt.savefig("results/plots/iGOATS_distributions.pdf")

In [ ]:
fig = plot_feature_combinations(
    jnp.concatenate([observations, actions], axis=-1),
    labels=["$\\tilde{i}_d$", "$\\tilde{i}_q$", "$\\tilde{u}_d$", "$\\tilde{u}_q$"]
);
plt.savefig("results/plots/iGOATS_data.pdf")

In [ ]:
from dmpe.utils.density_estimation import DensityEstimate
from dmpe.evaluation.plotting_utils import plot_2d_kde_as_contourf, plot_2d_kde_as_surface

In [ ]:
bw = 0.11

In [ ]:
obs_density = DensityEstimate.from_dataset(observations, actions=actions, use_actions=False, points_per_dim=100, bandwidth=bw)
obs_labels = ["i_d", "i_q"]
plot_2d_kde_as_contourf(obs_density.p, obs_density.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
act_density = DensityEstimate.from_dataset(actions, actions=observations, use_actions=False, points_per_dim=100, bandwidth=bw)

obs_labels = ["u_d", "u_q"]
plot_2d_kde_as_contourf(act_density.p, act_density.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
density_estimate_test =  DensityEstimate.from_dataset(jnp.concatenate([observations[..., 0][..., None], actions[..., 0][..., None]], axis=-1), actions=actions, use_actions=False, points_per_dim=100, bandwidth=bw)

obs_labels = ["i_d", "u_d"]
plot_2d_kde_as_contourf(density_estimate_test.p, density_estimate_test.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
density_estimate_test =  DensityEstimate.from_dataset(jnp.concatenate([observations[..., 1][..., None], actions[..., 1][..., None]], axis=-1), actions=actions, use_actions=False, points_per_dim=100, bandwidth=bw)

obs_labels = ["i_q", "u_q"]
plot_2d_kde_as_contourf(density_estimate_test.p, density_estimate_test.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
density_estimate_test =  DensityEstimate.from_dataset(jnp.concatenate([observations[..., 0][..., None], actions[..., 1][..., None]], axis=-1), actions=actions, use_actions=False, points_per_dim=100, bandwidth=bw)

obs_labels = ["i_d", "u_q"]
plot_2d_kde_as_contourf(density_estimate_test.p, density_estimate_test.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
density_estimate_test =  DensityEstimate.from_dataset(jnp.concatenate([observations[..., 1][..., None], actions[..., 0][..., None]], axis=-1), actions=actions, use_actions=False, points_per_dim=100, bandwidth=bw)

obs_labels = ["i_q", "u_d"]
plot_2d_kde_as_contourf(density_estimate_test.p, density_estimate_test.x_g, obs_labels)

plt.savefig("-".join(obs_labels) + ".png")

In [ ]:
from dmpe.related_work.mixed_GA import Permutation, Integer, Real

In [ ]:
action_dim = 3

prediction_horizon = 5
bounds_amplitude = (-1, 1)
bounds_duration = (5, 20)

amplitude_variables = {f"a_{number}": Real(bounds=bounds_amplitude) for number in range(prediction_horizon * action_dim)}  # 
duration_variables = {f"d_{number}": Integer(bounds=bounds_duration) for number in range(prediction_horizon * action_dim)}  # 
all_vars = dict(amplitude_variables, **duration_variables)

x = {name: var.sample() for name, var in all_vars.items()}

In [ ]:
action_parameters = np.fromiter(x.values(), dtype=np.float64)

amplitudes = action_parameters[:prediction_horizon * action_dim].reshape((action_dim, prediction_horizon))
durations = action_parameters[prediction_horizon * action_dim:].astype(np.int32).reshape((action_dim, prediction_horizon))

In [ ]:
def generate_aprbs(amplitudes, durations):
    """Parameterizable aprbs. This is used to transform the aprbs parameters into a signal."""
    return np.concatenate([np.ones(duration) * amplitude for (amplitude, duration) in zip(amplitudes, durations)])


def generate_multidim_aprbs(amplitudes, durations):
    assert amplitudes.shape == durations.shape
    min_length = np.min(np.sum(durations, axis=1))
    multidim_signal = np.concatenate([generate_aprbs(amplitude, duration)[:min_length][..., None] for (amplitude, duration) in zip(amplitudes, durations)], axis=-1)       
    return multidim_signal


def generate_mutlidim_aprbs_shared_durations(amplitudes, durations):
    assert amplitudes.shape[1] == durations.shape[0] and durations.ndim == 1
    multidim_signal = np.concatenate([generate_aprbs(amplitude, durations)[..., None] for amplitude in amplitudes], axis=-1)
    return multidim_signal

In [ ]:
plt.plot(generate_multidim_aprbs(amplitudes, durations))

In [ ]:
plt.plot(generate_mutlidim_aprbs_shared_durations(amplitudes, durations[0]))

In [ ]:
amplitudes.shape[0]

In [ ]:
amplitudes

In [ ]:
seed=0

env_params = dict(
    batch_size=1,
    tau=5,
    max_height=3,
    max_inflow=0.2,
    base_area=jnp.pi,
    orifice_area=jnp.pi * 0.1**2,
    c_d=0.6,
    g=9.81,
    env_solver=diffrax.Tsit5(),
)
env = excenvs.make(
    "FluidTank-v0",
    physical_constraints=dict(height=env_params["max_height"]),
    action_constraints=dict(inflow=env_params["max_inflow"]),
    static_params=dict(
        base_area=env_params["base_area"],
        orifice_area=env_params["orifice_area"],
        c_d=env_params["c_d"],
        g=env_params["g"],
    ),
    tau=env_params["tau"],
    solver=env_params["env_solver"],
)

h = 10
a = 10

alg_params = dict(
    prediction_horizon=h,
    application_horizon=a,
    bounds_amplitude=(-1, 1),
    bounds_duration=(5, 50),
    population_size=50,
    n_generations=25,
    featurize=lambda x: x,
    rng=None,
    compress_data=True,
    compression_target_N=500,
    rho_obs=1e3,
    rho_act=1e3,
    compression_feat_dim=-2,
    compression_dist_th=0.1,
)

exp_params = dict(
    n_timesteps=15000,
    seed=int(seed),
    alg_params=alg_params,
    env_params=env_params,
)

# run excitation algorithm
observations, actions = excite_with_iGOATS(
    n_timesteps=exp_params["n_timesteps"],
    env=env,
    prediction_horizon=alg_params["prediction_horizon"],
    application_horizon=alg_params["application_horizon"],
    bounds_amplitude=alg_params["bounds_amplitude"],
    bounds_duration=alg_params["bounds_duration"],
    population_size=alg_params["population_size"],
    n_generations=alg_params["n_generations"],
    featurize=alg_params["featurize"],
    rng=np.random.default_rng(seed),
    compress_data=alg_params["compress_data"],
    compression_target_N=alg_params["compression_target_N"],
    rho_obs=alg_params["rho_obs"],
    rho_act=alg_params["rho_act"],
    compression_feat_dim=alg_params["compression_feat_dim"],
    compression_dist_th=alg_params["compression_dist_th"],
    plot_subsequences=True,
)

In [ ]:
env_params = dict(batch_size=1, tau=2e-2, max_torque=5, g=9.81, l=1, m=1, env_solver=diffrax.Tsit5())
env = excenvs.make(
    env_id="Pendulum-v0",
    batch_size=env_params["batch_size"],
    action_constraints={"torque": env_params["max_torque"]},
    static_params={"g": env_params["g"], "l": env_params["l"], "m": env_params["m"]},
    solver=env_params["env_solver"],
    tau=env_params["tau"],
)

alg_params = dict(
    n_amplitudes=360,
    n_amplitude_groups=36,
    reuse_observations=True,
    bounds_duration=(10, 100),
    population_size=50,
    n_generations=25,
    featurize=lambda x: x,
    compress_data=True,
    compression_target_N=500,
    compression_dist_th=0.1,
    compression_feature_dim=-2,
    rho_obs=1e3,
    rho_act=1e3,
)

In [ ]:
seed = 0

exp_params = dict(
    seed=int(seed),
    alg_params=alg_params,
    env_params=env_params,
)

# setup PRNG
rng = np.random.default_rng(seed=seed)

# run excitation algorithm
observations, actions = excite_with_sGOATS(
    n_amplitudes=alg_params["n_amplitudes"],
    n_amplitude_groups=alg_params["n_amplitude_groups"],
    reuse_observations=alg_params["reuse_observations"],
    env=env,
    bounds_duration=alg_params["bounds_duration"],
    population_size=alg_params["population_size"],
    n_generations=alg_params["n_generations"],
    featurize=alg_params["featurize"],
    compress_data=alg_params["compress_data"],
    compression_target_N=alg_params["compression_target_N"],
    compression_dist_th=alg_params["compression_dist_th"],
    compression_feat_dim=alg_params["compression_feature_dim"],
    rho_obs=alg_params["rho_obs"],
    rho_act=alg_params["rho_act"],
    rng=np.random.default_rng(seed=exp_params["seed"]),
    verbose=False,
    plot_every_subsequence=True,
)

In [ ]:
env_params = dict(
    batch_size=1,
    tau=5e-1,
    max_height=3,
    max_inflow=0.2,
    base_area=jnp.pi,
    orifice_area=jnp.pi * 0.1**2,
    c_d=0.6,
    g=9.81,
    env_solver=diffrax.Euler(),
)
env = excenvs.make(
    "FluidTank-v0",
    physical_constraints=dict(height=env_params["max_height"]),
    action_constraints=dict(inflow=env_params["max_inflow"]),
    static_params=dict(
        base_area=env_params["base_area"],
        orifice_area=env_params["orifice_area"],
        c_d=env_params["c_d"],
        g=env_params["g"],
    ),
    tau=env_params["tau"],
    solver=env_params["env_solver"],
)

In [ ]:
prediction_horizon = 4
application_horizon = 4

igoats_observations, igoats_actions = excite_with_iGOATS(
    n_timesteps=15000,
    env=env,
    prediction_horizon=prediction_horizon,
    application_horizon=application_horizon,
    bounds_amplitude=[-1, 1],
    bounds_duration=[1, 100],
    population_size=50,
    n_generations=50,
    featurize=lambda x: x,
    rng=np.random.default_rng(0),
    compress_data=True,
    compression_target_N=500,
    rho_obs=1e3,
    rho_act=1e3,
    compression_feat_dim=-2,
    compression_dist_th=0.1,
    plot_subsequences=True,
)


In [ ]:
%debug

In [ ]:
plot_sequence(igoats_observations, igoats_actions, env.tau, env.obs_description, env.action_description)

In [ ]:
z = np.ones((51, 51))

In [ ]:
xx, yy = np.meshgrid(np.linspace(-1, 0, 51), np.linspace(-1, 1, 51))

In [ ]:
(xx + 1) * 25

In [ ]:
(xx**2 + yy**2 > 1).shape

In [ ]:
z[xx**2 + yy**2 > 1] = 0

In [ ]:
plt.imshow(z)

In [ ]:
dim = 3

points_per_dim = 100
n_grid_points = points_per_dim**dim

target_distribution = (np.ones(shape=[points_per_dim for _ in range(dim)]) ** dim)[..., None]

In [ ]:
target_distribution.shape

In [ ]:
xx, yy = np.meshgrid(np.linspace(-1, 0, points_per_dim), np.linspace(-1, 1, points_per_dim))
target_distribution[xx**2 + yy**2 > 1] = 0
target_distribution[yy < 0.1] = 0

In [ ]:
plt.imshow(target_distribution[:, :, 0])

In [ ]:
plt.imshow(target_distribution[:, 1, :])

In [ ]:
xx**2 + yy**2 > 1

In [ ]:
target_distribution[:, :, 0, 0]

In [ ]:
target_distribution[:, 0, :, 0]

In [ ]:
target_distribution[0, 0, :, :]